<a href="https://colab.research.google.com/github/Aifa-Chris/Cousera_Capstone/blob/master/Cousera_Toronto_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [65]:
import pandas as pd
import numpy as np
import requests

! pip install geocoder
import geocoder 



     |████████████████████████████████| 102kB 2.4MB/s 


In [59]:
data = "https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695"
Toronto = pd.read_html(data)[0]
Toronto.head(10)
# Make the dataframe (Using old html link)

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
9,M8A,Not assigned,Not assigned


In [60]:
Toronto1 = Toronto.replace('Not assigned', np.nan)
Toronto2 = Toronto1.dropna()
Toronto2
#Prepping data for dropna(), droping NaN

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
...,...,...,...
282,M8Z,Etobicoke,Kingsway Park South West
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West


In [81]:
Toronto3= Toronto2.groupby(['Postcode', 'Borough']).agg(lambda x:" , ".join(x))
Toronto3
#Grouping by Postcode, Borough

,,Neighbourhood
Postcode,Borough,
M1B,Scarborough,"Rouge , Malvern"
M1C,Scarborough,"Highland Creek , Rouge Hill , Port Union"
M1E,Scarborough,"Guildwood , Morningside , West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
...,...,...
M9N,York,Weston
M9P,Etobicoke,Westmount
M9R,Etobicoke,"Kingsview Village , Martin Grove Gardens , Ric..."


In [62]:
Toronto3.shape

(102, 1)

In [91]:
#Load coordinates, rename column to match
Coordinates = pd.read_csv('Geospatial_Coordinates.csv')
Coordinates1= Coordinates.rename(columns={'Postal Code':'Postcode'})
Coordinates1.head()

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [216]:
# Merge dataframes
Toronto4 = Toronto3.merge(Coordinates1, on='Postcode', how="left")
Toronto4

,Postcode,Neighbourhood,Latitude,Longitude
0,M1B,"Rouge , Malvern",43.806686,-79.194353
1,M1C,"Highland Creek , Rouge Hill , Port Union",43.784535,-79.160497
2,M1E,"Guildwood , Morningside , West Hill",43.763573,-79.188711
3,M1G,Woburn,43.770992,-79.216917
4,M1H,Cedarbrae,43.773136,-79.239476
...,...,...,...,...
97,M9N,Weston,43.706876,-79.518188
98,M9P,Westmount,43.696319,-79.532242
99,M9R,"Kingsview Village , Martin Grove Gardens , Ric...",43.688905,-79.554724
100,M9V,"Albion Gardens , Beaumond Heights , Humbergate...",43.739416,-79.588437


In [226]:
Search = Toronto4.loc[Toronto4.Neighbourhood == 'Church and Wellesley']
Search


ERROR! Session/line number was not unique in database. History logging moved to new session 70


,Postcode,Neighbourhood,Latitude,Longitude
52,M4Y,Church and Wellesley,43.66586,-79.38316


In [0]:
|import folium

In [0]:
Canada_map=folium.Map(location=[43.806686,-79.194353], zoom_start=8)
Canada_map


In [202]:
map_Toronto = folium.Map(location=[43.806686,-79.194353], zoom_start=10.5)

# add Neighbourhoods to map
for lat, lng, Neighbourhood in zip(Toronto4['Latitude'], Toronto4['Longitude'], Toronto4['Neighbourhood']):
    label = '{}'.format(Neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

In [203]:
CLIENT_ID = '1SHMANHM4J44TRJGHF1PXMBNUG0BDB5J2LJ3RN0VRYQPZZVP' # your Foursquare ID
CLIENT_SECRET = 'ORLLNBGEVE1I24KFEHUNFZUHE1MX54T1DRE5C2RYJIL3ANQI' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1SHMANHM4J44TRJGHF1PXMBNUG0BDB5J2LJ3RN0VRYQPZZVP
CLIENT_SECRET:ORLLNBGEVE1I24KFEHUNFZUHE1MX54T1DRE5C2RYJIL3ANQI


In [0]:
Toronto4.loc[53,'Neighbourhood']
neighbourhood_latitude=Toronto4.loc[53, 'Latitude']
neighbourhood_longitude=Toronto4.loc[53,'Longitude']
neighbourhood_name = Toronto4.loc[53,'Neighbourhood']
radius = 1000
LIMIT = 10

In [229]:
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=1SHMANHM4J44TRJGHF1PXMBNUG0BDB5J2LJ3RN0VRYQPZZVP&client_secret=ORLLNBGEVE1I24KFEHUNFZUHE1MX54T1DRE5C2RYJIL3ANQI&v=20180605&ll=43.6542599,-79.3606359&radius=1000&limit=10'

In [0]:
results = requests.get(url).json()


In [231]:
#Turning json into table 
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,Roselle Desserts,Bakery,43.653447,-79.362017
1,Tandem Coffee,Coffee Shop,43.653559,-79.361809
2,Cooper Koo Family YMCA,Distribution Center,43.653249,-79.358008
3,Impact Kitchen,Restaurant,43.656369,-79.356980
4,The Distillery Historic District,Historic Site,43.650244,-79.359323
5,Body Blitz Spa East,Spa,43.654735,-79.359874
6,Distillery Sunday Market,Farmers Market,43.650075,-79.361832
7,Rooster Coffee,Coffee Shop,43.651900,-79.365609
8,Corktown Common,Park,43.655618,-79.356211
9,Arvo,Coffee Shop,43.649963,-79.361442
